In [1]:
# autoreload magic
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from ray import tune

from mindreadingautobots.sequence_generators import make_datasets
from mindreadingautobots.models import rnn, hyperparameters  

In [24]:
# DATA LOADING
seed = 334
n_train = 800 
n_data = int(n_train * 5/4) # downstream we have a 80/20 train/val split
n_bits = 8 + 1
# k = 4
# noisy_not_majority_transition_matrix = {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.95, 4: 0.95}
# noisy_not_majority_transition_matrix = {0: 0.00, 1: 0.00, 2: 0.00, 3: 1, 4: 1}
k = 2
noisy_not_majority_transition_matrix = {0: 0.00, 1: 1, 2: 1}

X, _ = make_datasets.k_lookback_weight_dataset(noisy_not_majority_transition_matrix, k, n_data, n_bits, 0, seed)
# Insert feature dimension (1 for scalar bits), and convert to tensor
stochastic_majority_data = torch.tensor(X).float().unsqueeze(-1) 
print(stochastic_majority_data.shape)
config = {
        "hidden_size": 64,
        "num_layers": 6,
        "lr": 5e-5,
        "epochs": 1000,
        "n_eval": 4,
    } 


torch.Size([1000, 9, 1])


In [25]:
(model, hidden) = rnn.train_binary_rnn(config, stochastic_majority_data, verbose=True, return_model=True)


{'hidden_size': 64, 'num_layers': 6, 'lr': 5e-05, 'epochs': 1000, 'n_eval': 4}
{'loss': 0.6959856152534485, 'mean_accuracy': 0.2}
{'loss': 0.6422483325004578, 'mean_accuracy': 0.6888888888888889}
{'loss': 0.49859529733657837, 'mean_accuracy': 0.6888888888888889}
{'loss': 0.39164572954177856, 'mean_accuracy': 0.6888888888888889}
{'loss': 0.473330557346344, 'mean_accuracy': 0.6888888888888889}
{'loss': 0.44217056035995483, 'mean_accuracy': 0.7111111111111111}
{'loss': 0.3741046190261841, 'mean_accuracy': 0.7555555555555555}
{'loss': 0.33776533603668213, 'mean_accuracy': 0.7777777777777778}
{'loss': 0.34727007150650024, 'mean_accuracy': 0.7666666666666667}
{'loss': 0.3648398816585541, 'mean_accuracy': 0.7666666666666667}
{'loss': 0.42478519678115845, 'mean_accuracy': 0.7222222222222222}
{'loss': 0.35592901706695557, 'mean_accuracy': 0.7666666666666667}
{'loss': 0.3906911313533783, 'mean_accuracy': 0.7333333333333333}
{'loss': 0.26873207092285156, 'mean_accuracy': 0.8333333333333334}
{'los

KeyboardInterrupt: 

In [12]:
X.shape

(1000, 9)

In [21]:
xx = stochastic_majority_data[:10,:-1]
yy = stochastic_majority_data[:10,1:]
print(xx.reshape(10, -1))
preds = model.predict(xx, hidden).reshape(10, -1)
print(preds)
print(yy.reshape(10, -1))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1.]])
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.,